In [8]:
import ast
import tokenize
from io import BytesIO
from typing import List, Dict, Union, Optional, Any, Type, Callable
from typing_extensions import TypedDict
from rich import print

from file import get_file_paths

PROJECT_DIR = "./MetaGPT/metagpt"
SUPPORTED_FILE_TYPES = [".py"]

class DependencyData(TypedDict):
    modules: List[str]
    functions_called: List[str]
    global_vars: List[str]
    classes_used: List[str]
    function_codes: Dict[str, str]
    class_codes: Dict[str, str]
    
def extract_from_node(extractor: Type, node: ast.AST) -> List[str]:
    return extractor.extract(node)

class ImportExtractor:
    @staticmethod
    def extract(node: ast.AST) -> List[str]:
        if isinstance(node, ast.Import):
            return [n.name for n in node.names]
        elif isinstance(node, ast.ImportFrom):
            return [f"{node.module}.{n.name}" for n in node.names]
        return []

class FunctionCallExtractor:
    @staticmethod
    def extract(node: ast.AST) -> List[str]:
        if isinstance(node, ast.Call):
            if isinstance(node.func, ast.Name):
                return [node.func.id]
            elif isinstance(node.func, ast.Attribute):
                return [node.func.attr]
        return []

class GlobalVarExtractor:
    @staticmethod
    def extract(node: ast.AST) -> List[str]:
        return node.names if isinstance(node, ast.Global) else []

class ClassUsageExtractor:
    @staticmethod
    def extract(node: ast.AST) -> List[str]:
        return [node.name] if isinstance(node, ast.ClassDef) else []

def extract_code_segment(code: str, node: Union[ast.FunctionDef, ast.ClassDef]) -> Optional[str]:
    # Use the line numbers provided by the ast node
    start_line = node.lineno - 1  # AST line numbers are 1-based, lists are 0-based
    end_line = node.end_lineno if hasattr(node, 'end_lineno') else start_line
    return '\n'.join(code.splitlines()[start_line:end_line + 1])

def analyze_code(code: str) -> DependencyData:
    tree = ast.parse(code)
    
    data: DependencyData = {
        'modules': [],
        'functions_called': [],
        'global_vars': [],
        'classes_used': [],
        'function_codes': {},
        'class_codes': {}
    }

    extractors = {
        'modules': ImportExtractor,
        'functions_called': FunctionCallExtractor,
        'global_vars': GlobalVarExtractor,
        'classes_used': ClassUsageExtractor
    }

    for key, extractor in extractors.items():
        for node in ast.walk(tree):
            data[key].extend(extract_from_node(extractor, node))

    for node in tree.body:  # Only top-level nodes
        if isinstance(node, ast.FunctionDef) and not node.name.startswith('_'):
            data['function_codes'][node.name] = extract_code_segment(code, node)
        elif isinstance(node, ast.ClassDef):
            data['class_codes'][node.name] = extract_code_segment(code, node)

    return data



def main():
    paths = get_file_paths(PROJECT_DIR, SUPPORTED_FILE_TYPES)
    
    for path in paths:
        with open(path, "r") as source:
            code = source.read()
        
        result = analyze_code(code)
        print(result['class_codes'])
        break


if __name__ == "__main__":
    main()

class NotConfiguredException(Exception):
    """Exception raised for errors in the configuration.

    Attributes:
        message -- explanation of the error
    """

    def __init__(self, message="The required configuration is not set"):
        self.message = message
        super().__init__(self.message)